In [1]:
import sys
sys.path.insert(0, "/notebooks/pipenv")
sys.path.insert(0, "/notebooks/nebula3_database")
sys.path.insert(0, "/notebooks/nebula3_experiments")
sys.path.insert(0, "/notebooks/nebula3_videoprocessing")
sys.path.insert(0, "/notebooks/")
from PIL import Image
import requests
import visual_genome.local as vg
import json
import copy
import operator
import itertools
import subprocess

import numpy as np
import torch
import spacy
import nltk
import openai
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
from sentence_transformers import SentenceTransformer
from database.arangodb import DatabaseConnector
from config import NEBULA_CONF
from vg_eval import VGEvaluation, get_sc_graph, spice_get_triplets, tuples_from_sg
from videoprocessing.vlm_factory import VlmFactory
from videoprocessing.vlm_interface import VlmInterface
from videoprocessing.vlm_implementation import VlmChunker

In [2]:
nltk.download('wordnet')
nlp = spacy.load('en_core_web_lg')
# nlp.add_pipe("spacy_wordnet", after='tagger', config={'lang': nlp.lang})

with open('/storage/keys/openai.key','r') as f:
    OPENAI_API_KEY = f.readline().strip()
openai.api_key = OPENAI_API_KEY

VG_DATA = '/storage/vg_data'
IPC_COLLECTION = 'ipc_relations_spice'
RECALL_COLLECTION = 'ipc_recall_spice'
GLOBAL_TOKENS_COLLECTION = 's3_global_tokens'
S3_PARAGRAPH_1_COLLECTION = 's3_paragraph_1'
S3_RECALL_1_COLLECTION = 's3_recall_1'
FS_GPT_MODEL = 'text-davinci-002'
FS_SAMPLES = 5                   # Samples for few-shot gpt
class PIPELINE:
    def __init__(self):
        config = NEBULA_CONF()
        self.db_host = config.get_database_host()
        self.database = config.get_playground_name()
        self.gdb = DatabaseConnector()
        self.db = self.gdb.connect_db(self.database)
pipeline = PIPELINE()
def get_all_s3_ids():
    results = {}
    query = 'FOR doc IN {} RETURN doc.image_id'.format(GLOBAL_TOKENS_COLLECTION)
    cursor = pipeline.db.aql.execute(query)
    return [doc for doc in cursor]

s3_ids = get_all_s3_ids()
evaluator = VGEvaluation()
def flatten(lst): return [x for l in lst for x in l]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
def get_image_id_from_collection(id,collection=GLOBAL_TOKENS_COLLECTION):
    results = {}
    query = 'FOR doc IN {} FILTER doc.image_id == {} RETURN doc'.format(collection,id)
    cursor = pipeline.db.aql.execute(query)
    for doc in cursor:
        results.update(doc)
    return results

class GTBaseGenerator:
    def __init__(self):
        self.pipeline = PIPELINE()
        self.ipc_data = json.load(open('/storage/ipc_data/paragraphs_v1.json','r'))
        self.global_captioner = 'blip'
        self.global_tagger = 'blip'
        self.places_source = 'blip'
        self.global_prompt1 = '''Caption of image: {}
This image is taking place in: {}
Tags: This image is about {}
Describe this image in detail:'''

    
    def get_structure(self, id):
        sg = get_sc_graph(id)
        global_doc = get_image_id_from_collection(id)
        if not global_doc:
            print("Couldn't find global tokens for id {}".format(id))
            return
        rc_doc = {
            'image_id': id,
            'url': sg.image.url            
        }
        for (k,v) in global_doc.items():
            if k.startswith('global'):
                rc_doc[k]=copy.copy(v)
        rois = []
        for obj in sg.objects:
            obj_dic = {
                'GT': list(zip(obj.names,[1.0]*len(obj.names)))
            }
            attr_dic = {
                'GT': list(zip(obj.attributes,[1.0]*len(obj.attributes)))
            }
            obj_doc = {                
                'objects': obj.names,
                'attributes': obj.attributes,
                'bbox': [obj.x, obj.y, obj.x+obj.width, obj.y+obj.height]              
                }
            rois.append(obj_doc)
        rc_doc['rois']=rois

        return rc_doc

    def get_prompt(self, id, include_answer=False):
        base_doc = self.get_structure(id)
        if base_doc == None:
            return
        caption = base_doc['global_captions'][self.global_captioner]
        all_objects = base_doc['global_objects'][self.global_tagger]
        all_persons = base_doc['global_persons'][self.global_tagger]
        all_places = base_doc['global_scenes'][self.places_source]
        # print("Caption: {}".format(caption))
        # print("Objects: ")
        # print(all_objects[:5])
        # print("Places:")
        # print(all_places[:5])
        # print("Persons:")
        # print(all_persons[:5])
        objects = '; '.join([x['label'] for x in all_objects[:8]])
        persons = '; '.join([x['label'] for x in all_persons[:5]])
        places = ' or '.join([x['label'] for x in all_places[:3]])
        prompt_before_answer = self.global_prompt1.format(caption,places,objects)
        if include_answer:
            [answer] = [x['paragraph'] for x in self.ipc_data if x['image_id']==id]
            final_prompt = prompt_before_answer+" "+answer
        else:
            final_prompt = prompt_before_answer
        return final_prompt
        
base_gen = GTBaseGenerator()

def few_shot_process_target_id(fs_ids: list[int],target_id: int, vlm: VlmInterface, pgen=base_gen, **kwargs):
    target_sg = get_sc_graph(target_id)
    fs_prompt = generate_gpt_prompt(fs_ids, target_id=target_id, pgen=pgen)
    results = gpt_execute(fs_prompt, model=FS_GPT_MODEL, **kwargs)
    return results
    # scores = vlm.compute_similarity_url(target_sg.image.url,results)
    # best_index = np.argmax(scores)
    # return results[best_index]


In [4]:
def gpt_execute(prompt_template, *args, **kwargs):            
    prompt = prompt_template.format(*args)   
    response = openai.Completion.create(prompt=prompt, max_tokens=256, **kwargs)   
    # return response
    return [x['text'].strip() for x in response['choices']]

def generate_gpt_prompt(ids, target_id=None, pgen=GTBaseGenerator()):
    rc = []
    for id in ids:
        rc.append(pgen.get_prompt(id,include_answer=True))
    if target_id:
        rc.append(pgen.get_prompt(target_id,include_answer=False))
    return '\n'.join(rc)


In [5]:
def candidates_from_paragraph(paragraph):
    senter = nlp.get_pipe("senter")
    sentences = [str(x) for x in senter(nlp(paragraph)).sents]
    n = len(sentences)
    cands = []
    for i in range(3,n+1):
        for comb in itertools.combinations(range(n),i):
            cands.append(' '.join(operator.itemgetter(*comb)(sentences)))
    return cands

In [6]:
vlm = VlmChunker(VlmFactory().get_vlm("blip_itc"), chunk_size=5)

Initializing model on GPU
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_retrieval_coco.pth


In [7]:
s3_train, s3_test = np.split(np.array(s3_ids),[800])

In [8]:
# train_ids = np.random.choice(s3_train,5)
# target_id = np.random.choice(s3_test)
# # rc = generate_gpt_prompt(train_ids, target_id=target_id, pgen=base_gen)
# rc = few_shot_process_target_id(train_ids,target_id,vlm,n=5)
# candidates = flatten([candidates_from_paragraph(x) for x in rc])

In [18]:
def process_target_id(target_id, train_ids, vlm, n=5, **kwargs):
    sg = get_sc_graph(target_id)
    print("Processing target_id {}, url: {}".format(target_id,sg.image.url))
    train_ids = np.random.choice(s3_train,FS_SAMPLES)
    rc = few_shot_process_target_id(train_ids, target_id, vlm, n=n, **kwargs)
    candidates = flatten([candidates_from_paragraph(x) for x in rc])
    scores = vlm.compute_similarity_url(sg.image.url,candidates)
    cand = candidates[np.argmax(scores)]
    return {
        'image_id': target_id,
        'url': sg.image.url,
        'paragraphs': rc,
        'candidate': cand
    }

def process_recall(target_id):
    doc = get_image_id_from_collection(target_id,S3_PARAGRAPH_1_COLLECTION)
    if not doc:
        print("Paragraph document for target id {} missing. Aborting".format(target_id))
        return
    sg = get_sc_graph(target_id)
    gt_triplets = tuples_from_sg(sg)
    pred_triplets = spice_get_triplets(doc['candidate'])
    # print("Ground Triplets:")
    # print(gt_triplets)
    # print("Pred triplets:")
    # print(pred_triplets)
    recall = evaluator.recall_triplets_mean(gt_triplets,pred_triplets)
    precision = evaluator.recall_triplets_mean(pred_triplets,gt_triplets)
    return {
        'image_id': target_id,
        'mean_recall': recall,
        'mean_precision': precision
    }


def process_gen_paragraph_all_ids(train_ids, target_ids, vlm, **kwargs):
    for target_id in target_ids:
        rc = get_image_id_from_collection(target_id,S3_PARAGRAPH_1_COLLECTION)
        if rc:
            print("target id {} already exists.".format(target_id))
            continue

        print("Moving on with target id {}.".format(target_id))
        rc_doc = process_target_id(target_id, train_ids, vlm, **kwargs)
        query = "INSERT {} INTO {}".format(rc_doc,S3_PARAGRAPH_1_COLLECTION)
        cursor = pipeline.db.aql.execute(query)  

def process_recall_all_ids(target_ids):
    for target_id in target_ids:
        rc = get_image_id_from_collection(target_id,S3_RECALL_1_COLLECTION)
        if rc:
            print("target id {} already exists.".format(target_id))
            continue

        print("Moving on with target id {}.".format(target_id))
        rc_doc = process_recall(target_id)
        query = "INSERT {} INTO {}".format(rc_doc,S3_RECALL_1_COLLECTION)
        cursor = pipeline.db.aql.execute(query)  
         


In [ ]:
process_gen_paragraph_all_ids(s3_train,s3_test,vlm,n=5)

In [21]:
process_recall_all_ids(s3_test)

target id 2370676 already exists.
target id 2402584 already exists.
target id 2390591 already exists.
Moving on with target id 2334467.
Moving on with target id 2405264.
Moving on with target id 2345862.
Moving on with target id 2322127.
Moving on with target id 2345047.
Moving on with target id 2400355.
Moving on with target id 4550.
Moving on with target id 2366505.
Moving on with target id 2340989.
Moving on with target id 2412473.
Moving on with target id 2407398.
Moving on with target id 2396279.
Moving on with target id 2343486.
Moving on with target id 2348753.
Moving on with target id 2356333.
Moving on with target id 2342746.
Moving on with target id 2394355.
Moving on with target id 2371761.
Moving on with target id 2412770.
Moving on with target id 2388946.
Moving on with target id 2373834.
Moving on with target id 2360872.
Moving on with target id 2401158.
Moving on with target id 2263.
Moving on with target id 2372687.
Moving on with target id 2322096.
Moving on with targe

In [13]:
rc

{'image_id': 2402584, 'mean_recall': 0.5702875, 'mean_precision': 0.68995017}

In [ ]:
sg = get_sc_graph(target_id)
print(sg.image.url)
gt_triplets = tuples_from_sg(sg)
pred_triplets = spice_get_triplets(gpt_rc[0])
recall = evaluator.recall_triplets_mean(gt_triplets,pred_triplets)
print("Mean (bert-based) total recall of ground truth triplets in ipc triplets is: {}".format(recall))


In [ ]:
sg = get_sc_graph(2359808)

In [ ]:
[(x.x, x.y, x.x+x.width, x.y+x.height) for x in sg.objects if x.width>=50 and x.height>=50]

In [ ]:
rc = senter(nlp("Hello world. Hello world again."))

In [ ]:
list(rc.sents)